In [542]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.genmod.generalized_estimating_equations import GEE , GLM
from statsmodels.genmod.cov_struct import Independence
from statsmodels.genmod.families import Poisson

In [543]:
!pip install pyreadstat

In [544]:
df = pd.read_csv("Team Assignment 1 Data.csv", sep=";")

In [545]:
df.shape

(600, 23)

In [546]:
df.describe(include='all')

,Review_ID,Number_Words_Review,ZNumber_Words_Review,Prod_Desig,Prod_Design_positive,Prod_Design_negative,Prod_Technical,Prod_Technical_positive,Prod_Technical_negative,Prod_Price,...,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases
count,600.000000,600.000000,600,600.000000,600.000000,600.000000,600.000000,600.000000,600.00000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600,600.000000,600.000000,600.000000,600.000000
unique,NaN,NaN,126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,600,NaN,NaN,NaN,NaN
top,NaN,NaN,"-0,601536598",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"54,52101351",NaN,NaN,NaN,NaN
freq,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
mean,300.500000,41.615000,NaN,0.655000,0.483333,0.311667,0.855000,0.720000,0.27000,0.256667,...,0.040000,0.020000,1.000000,0.500000,0.436667,NaN,3.691667,4.190000,0.093333,0.410000
std,173.349358,42.582613,NaN,0.475765,0.500139,0.463561,0.352395,0.449374,0.44433,0.437158,...,0.196123,0.140117,0.817178,0.500417,0.496386,NaN,1.248678,1.239541,0.291142,0.626542
min,1.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,0.000000,0.000000
25%,150.750000,16.000000,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,3.000000,4.000000,0.000000,0.000000
50%,300.500000,28.000000,NaN,1.000000,0.000000,0.000000,1.000000,1.000000,0.00000,0.000000,...,0.000000,0.000000,1.000000,0.500000,0.000000,NaN,4.000000,5.000000,0.000000,0.000000
75%,450.250000,55.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,0.000000,0.000000,2.000000,1.000000,1.000000,NaN,5.000000,5.000000,0.000000,1.000000


In [547]:
df.head(10)

,Review_ID,Number_Words_Review,ZNumber_Words_Review,Prod_Desig,Prod_Design_positive,Prod_Design_negative,Prod_Technical,Prod_Technical_positive,Prod_Technical_negative,Prod_Price,...,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases
0,1,34,"-0,178828858",1,1,0,1,1,0,0,...,0,0,0,1,0,"24,56327497",4,5,1,1
1,2,32,"-0,225796385",1,1,0,1,1,0,1,...,0,0,0,1,0,"38,27120469",5,4,0,0
2,3,54,"0,290846409",1,0,1,1,0,1,1,...,0,1,0,1,0,"46,77239881",2,2,0,0
3,4,20,"-0,507601545",1,1,0,1,1,0,0,...,0,0,0,1,0,"31,66539161",5,5,1,2
4,5,12,"-0,695471652",1,1,0,1,1,0,0,...,0,0,0,1,0,"25,31939178",3,5,0,0
5,6,29,"-0,296247675",0,0,0,0,0,0,1,...,0,1,0,1,0,"50,87669151",2,1,0,0
6,7,21,"-0,484117781",1,1,0,1,1,0,1,...,0,0,0,1,1,"34,27604718",5,5,0,0
7,8,10,"-0,742439178",0,0,0,1,1,0,0,...,0,0,0,1,1,"38,94502717",4,5,0,0
8,9,3,"-0,906825522",0,0,0,1,1,0,1,...,0,0,0,1,1,"20,46355334",5,5,0,0
9,10,78,"0,854456729",1,1,0,1,1,0,0,...,0,0,0,1,1,"22,85047959",5,5,1,2


In [548]:
df.drop(['Prod_Desig', 'Prod_Technical', 'Prod_Price', 'Serv_Delivery', 'ZNumber_Words_Review'], axis=1, inplace=True)


In [549]:
# Transform from decimals to float, converting , to .
df['Age'] = df['Age'].str.replace(',', '.').astype(float)
#df['ZNumber_Words_Review'] = df['ZNumber_Words_Review'].str.replace(',', '.').astype(float)

In [550]:
# create a new column with AGE in ranges from 18-35, 36-65
df['Age_Range'] = pd.cut(df['Age'], bins=[18, 35, 65], labels=['18-35', '36-65'])
df.head()

,Review_ID,Number_Words_Review,Prod_Design_positive,Prod_Design_negative,Prod_Technical_positive,Prod_Technical_negative,Prod_Price_positive,Prod_Price_negative,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Age_Range
0,1,34,1,0,1,0,0,0,0,0,0,1,0,24.563275,4,5,1,1,18-35
1,2,32,1,0,1,0,1,0,0,0,0,1,0,38.271205,5,4,0,0,36-65
2,3,54,0,1,0,1,0,1,0,1,0,1,0,46.772399,2,2,0,0,36-65
3,4,20,1,0,1,0,0,0,0,0,0,1,0,31.665392,5,5,1,2,18-35
4,5,12,1,0,1,0,0,0,0,0,0,1,0,25.319392,3,5,0,0,18-35


In [551]:
# transform AGE_RANGE to dummies
df = pd.get_dummies(df, columns=['Age_Range'], drop_first=True)
df.head()

,Review_ID,Number_Words_Review,Prod_Design_positive,Prod_Design_negative,Prod_Technical_positive,Prod_Technical_negative,Prod_Price_positive,Prod_Price_negative,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Age_Range_36-65
0,1,34,1,0,1,0,0,0,0,0,0,1,0,24.563275,4,5,1,1,0
1,2,32,1,0,1,0,1,0,0,0,0,1,0,38.271205,5,4,0,0,1
2,3,54,0,1,0,1,0,1,0,1,0,1,0,46.772399,2,2,0,0,1
3,4,20,1,0,1,0,0,0,0,0,0,1,0,31.665392,5,5,1,2,0
4,5,12,1,0,1,0,0,0,0,0,0,1,0,25.319392,3,5,0,0,0


In [552]:
df.drop(["Age"], axis=1, inplace=True)
df.head()

,Review_ID,Number_Words_Review,Prod_Design_positive,Prod_Design_negative,Prod_Technical_positive,Prod_Technical_negative,Prod_Price_positive,Prod_Price_negative,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Age_Range_36-65
0,1,34,1,0,1,0,0,0,0,0,0,1,0,4,5,1,1,0
1,2,32,1,0,1,0,1,0,0,0,0,1,0,5,4,0,0,1
2,3,54,0,1,0,1,0,1,0,1,0,1,0,2,2,0,0,1
3,4,20,1,0,1,0,0,0,0,0,0,1,0,5,5,1,2,0
4,5,12,1,0,1,0,0,0,0,0,0,1,0,3,5,0,0,0


In [553]:
df['Utilitarian'] = np.where((df['Category'] == 2) | (df['Category'] == 0), 1, 0)
df['Hedonic'] = np.where((df['Category'] == 2) | (df['Category'] == 1), 1, 0)
df.head()

,Review_ID,Number_Words_Review,Prod_Design_positive,Prod_Design_negative,Prod_Technical_positive,Prod_Technical_negative,Prod_Price_positive,Prod_Price_negative,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Age_Range_36-65,Utilitarian,Hedonic
0,1,34,1,0,1,0,0,0,0,0,0,1,0,4,5,1,1,0,1,0
1,2,32,1,0,1,0,1,0,0,0,0,1,0,5,4,0,0,1,1,0
2,3,54,0,1,0,1,0,1,0,1,0,1,0,2,2,0,0,1,1,0
3,4,20,1,0,1,0,0,0,0,0,0,1,0,5,5,1,2,0,1,0
4,5,12,1,0,1,0,0,0,0,0,0,1,0,3,5,0,0,0,1,0


In [554]:
df.drop(["Category"], axis=1, inplace=True)
df.head()

,Review_ID,Number_Words_Review,Prod_Design_positive,Prod_Design_negative,Prod_Technical_positive,Prod_Technical_negative,Prod_Price_positive,Prod_Price_negative,Serv_Delivery_positive,Serv_Delivery_negative,Country,Gender,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Age_Range_36-65,Utilitarian,Hedonic
0,1,34,1,0,1,0,0,0,0,0,1,0,4,5,1,1,0,1,0
1,2,32,1,0,1,0,1,0,0,0,1,0,5,4,0,0,1,1,0
2,3,54,0,1,0,1,0,1,0,1,1,0,2,2,0,0,1,1,0
3,4,20,1,0,1,0,0,0,0,0,1,0,5,5,1,2,0,1,0
4,5,12,1,0,1,0,0,0,0,0,1,0,3,5,0,0,0,1,0


In [555]:
columns1 = df.columns.tolist()
print(columns1)

['Review_ID', 'Number_Words_Review', 'Prod_Design_positive', 'Prod_Design_negative', 'Prod_Technical_positive', 'Prod_Technical_negative', 'Prod_Price_positive', 'Prod_Price_negative', 'Serv_Delivery_positive', 'Serv_Delivery_negative', 'Country', 'Gender', 'Sentiment', 'Rating_Score', 'Purchase', 'Number_of_Purchases', 'Age_Range_36-65', 'Utilitarian', 'Hedonic']


In [556]:
columns1 = df.columns.tolist()
columns1.remove('Rating_Score')
columns1.remove('Review_ID')
X1 = df[columns1]
print(X1)

     Number_Words_Review  Prod_Design_positive  Prod_Design_negative  \
0                     34                     1                     0   
1                     32                     1                     0   
2                     54                     0                     1   
3                     20                     1                     0   
4                     12                     1                     0   
..                   ...                   ...                   ...   
595                   38                     1                     1   
596                   65                     1                     1   
597                   79                     1                     1   
598                   14                     1                     0   
599                   93                     1                     1   

     Prod_Technical_positive  Prod_Technical_negative  Prod_Price_positive  \
0                          1                        0    

Y1 = df['Rating_Score']
print(Y1)

In [557]:
Y1 = df['Rating_Score']
print(Y1)

0      5
1      4
2      2
3      5
4      5
      ..
595    5
596    5
597    4
598    4
599    5
Name: Rating_Score, Length: 600, dtype: int64


In [558]:
X1 = sm.add_constant(X1)
model = sm.OLS(Y1, X1).fit()
predictions = model.predict(X1)
print_model = model.summary()
print(print_model)


                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     74.42
Date:                Wed, 20 Mar 2024   Prob (F-statistic):          1.56e-133
Time:                        18:11:18   Log-Likelihood:                -633.23
No. Observations:                 600   AIC:                             1302.
Df Residuals:                     582   BIC:                             1382.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [559]:
columns2 = df.columns.tolist()
columns2.remove('Purchase')
columns2.remove('Review_ID')
X2 = df[columns2]
print(X2)

     Number_Words_Review  Prod_Design_positive  Prod_Design_negative  \
0                     34                     1                     0   
1                     32                     1                     0   
2                     54                     0                     1   
3                     20                     1                     0   
4                     12                     1                     0   
..                   ...                   ...                   ...   
595                   38                     1                     1   
596                   65                     1                     1   
597                   79                     1                     1   
598                   14                     1                     0   
599                   93                     1                     1   

     Prod_Technical_positive  Prod_Technical_negative  Prod_Price_positive  \
0                          1                        0    

In [560]:
Y2 = df['Purchase']
print(Y2)

0      1
1      0
2      0
3      1
4      0
      ..
595    1
596    0
597    0
598    0
599    0
Name: Purchase, Length: 600, dtype: int64


In [561]:
X2 = sm.add_constant(X2)
logit_mod = sm.Logit(Y2, X2) # If the dependent variable is considered to be a truly qualitative and binomial character then LOGIT modelling should be preferred
logit_res = logit_mod.fit()
print(logit_res.summary())

         Current function value: 0.569741
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:               Purchase   No. Observations:                  600
Model:                          Logit   Df Residuals:                      582
Method:                           MLE   Df Model:                           17
Date:                Wed, 20 Mar 2024   Pseudo R-squ.:                 -0.8368
Time:                        18:11:19   Log-Likelihood:                -341.84
converged:                      False   LL-Null:                       -186.11
Covariance Type:            nonrobust   LLR p-value:                     1.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     -12.4571      2.347     -5.307      0.000     -17.058      -7.856
Number_Words_Review       

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [562]:
columns3 = df.columns.tolist()
columns3.remove('Number_of_Purchases')
X3 = df[columns3]
print(X3)

     Review_ID  Number_Words_Review  Prod_Design_positive  \
0            1                   34                     1   
1            2                   32                     1   
2            3                   54                     0   
3            4                   20                     1   
4            5                   12                     1   
..         ...                  ...                   ...   
595        596                   38                     1   
596        597                   65                     1   
597        598                   79                     1   
598        599                   14                     1   
599        600                   93                     1   

     Prod_Design_negative  Prod_Technical_positive  Prod_Technical_negative  \
0                       0                        1                        0   
1                       0                        1                        0   
2                       1     

In [563]:
Y3 = df['Number_of_Purchases']
print(Y3)

0      1
1      0
2      0
3      2
4      0
      ..
595    1
596    0
597    0
598    0
599    0
Name: Number_of_Purchases, Length: 600, dtype: int64


In [564]:
fam = Poisson()
ind = Independence()
model = sm.GEE(Y3, X3, groups=df['Review_ID'], cov_struct=ind, family=fam)
result = model.fit()
print(result.summary())

                               GEE Regression Results                              
Dep. Variable:         Number_of_Purchases   No. Observations:                  600
Model:                                 GEE   No. clusters:                      600
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                            Poisson   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     1
Date:                     Wed, 20 Mar 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         18:11:19
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Review_ID                   0.0006      0.000      1.569    

In [565]:
# Create the interaction term
df['Product_Design_Sentiment_Interact'] = df['Prod_Design_positive'] * df['Sentiment']

# Define the independent variables including the interaction term
X = df[['Prod_Design_positive', 'Sentiment', 'Product_Design_Sentiment_Interact']]

# Add a constant term for the intercept
X = sm.add_constant(X)

# Define the dependent variable
Y = df['Rating_Score']

# Fit the linear regression model
model = sm.OLS(Y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     347.6
Date:                Wed, 20 Mar 2024   Prob (F-statistic):          1.95e-130
Time:                        18:11:19   Log-Likelihood:                -676.27
No. Observations:                 600   AIC:                             1361.
Df Residuals:                     596   BIC:                             1378.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [566]:
# Create the interaction term
df['Country_Age_Interact'] = df['Country'] * df['Age_Range_36-65']

# Define the independent variables including the interaction term
X = df[['Country', 'Age_Range_36-65', 'Country_Age_Interact']]

# Add a constant term for the intercept
X = sm.add_constant(X)

# Define the dependent variable
Y = df['Rating_Score']

# Fit the linear regression model
model = sm.OLS(Y, X).fit()

# Print the summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     10.12
Date:                Wed, 20 Mar 2024   Prob (F-statistic):           1.64e-06
Time:                        18:11:19   Log-Likelihood:                -964.80
No. Observations:                 600   AIC:                             1938.
Df Residuals:                     596   BIC:                             1955.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.4902 